In [1]:

from config import GlobalConfig, tokenizer_parse_args
from helpers.Data import TokensPaths, load_queries, load_dictionary
from transformers import AutoTokenizer
import json


c:\Users\antoi\Desktop\thesis\biosyn_faiss_prod\myenc\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = GlobalConfig()
dictionary_max_length = cfg.tokenize.dictionary_max_length
dictionary_max_chars_length = cfg.tokenize.dictionary_max_chars_length
queries_max_length = cfg.tokenize.queries_max_length
mention_start_special_token = cfg.tokenize.special_tokens_dict["mention_start"]
mention_end_special_token = cfg.tokenize.special_tokens_dict["mention_end"]
tokens_paths  = TokensPaths(cfg, dictionary_key='dictionary', queries_key='train_queries')

tokenizer = AutoTokenizer.from_pretrained(cfg.model.model_name, use_fast=True)
tokenizer.add_special_tokens(cfg.tokenize.special_tokens)

mention_start_token_id  = tokenizer.convert_tokens_to_ids(cfg.tokenize.special_tokens_dict["mention_start"])
mention_end_token_id  = tokenizer.convert_tokens_to_ids(cfg.tokenize.special_tokens_dict["mention_end"])




In [3]:
train_queries = load_queries(
    data_dir=cfg.paths.queries_raw_dir,
    queries_max_length=queries_max_length,
    special_token_start=mention_start_special_token ,
    special_token_end=mention_end_special_token,
    tokenizer=tokenizer)

100%|██████████| 691/691 [00:01<00:00, 360.22it/s]

annotation_skipped: 13


In [4]:
from tokenizer import tokenize_names
from datasets import Dataset
import os

max_length = cfg.tokenize.queries_max_length
queries_names = [q[0] for q in train_queries]
queries_cuis = [q[1] for q in train_queries]
queries_sentences = [q[2] for q in train_queries]

N = len(queries_sentences)
tokenized = tokenizer(queries_sentences, padding="max_length", truncation=True, max_length=max_length)




In [5]:
import torch

inps_all  = torch.as_tensor(tokenized["input_ids"])
atts_all  = torch.as_tensor(tokenized["attention_mask"])

In [6]:
idx = 4599
st = str(train_queries[idx][2])
len(tokenizer(st)["input_ids"])



73

In [7]:
len(tokenized["input_ids"][4599])



75

In [ ]:


mention_start_token_id  = tokenizer.convert_tokens_to_ids(cfg.tokenize.special_tokens_dict["mention_start"])
mention_end_token_id  = tokenizer.convert_tokens_to_ids(cfg.tokenize.special_tokens_dict["mention_end"])
inps_all = dataset.queries_input_ids
for idx, inp  in enumerate(inps_all):
    mention_start_position = (inp == mention_start_token_id).nonzero()[0], f"query_idx: {idx}, does not have start: inps: {inp}"
    assert len(mention_start_position) > 0
    mention_end_position = (inp == mention_end_token_id).nonzero()[0], f"query_idx: {idx}, does not have end: inps: {inp}"
    assert len(mention_end_position) > 0

mention_end_position

tensor([15])

In [ ]:
tokenizer(e["text"], padding="max_length", truncation=True, max_length=75)
mention_start_positions = (inp == mention_start_token_id).nonzero()[0]

In [ ]:


(tokens_size, max_length ) = tokens_paths.queries_shape
batch_size = 128
N = tokens_size

query_inputs = dataset.queries_input_ids
query_att = dataset.queries_attention_mask

device = "cuda"

start = 14719
end = 14849
cursor = 0

for i in range(start, end):
    print(f"idx: {i}")
    mention_start_positions = (query_inputs[i] == mention_start_token_id).nonzero()[0]
    mention_end_positions = (query_inputs[i] == mention_end_token_id).nonzero()[0]





In [ ]:
(tokens_size, max_length ) = tokens_paths.queries_shape
N = tokens_size

query_inputs = dataset.queries_input_ids
query_att = dataset.queries_attention_mask



for start in range(0, N,batch_size):
    end = min(start + batch_size, N)
    print(f"start:end: {start}-{end}")
    inp  = torch.as_tensor(query_inputs[start:end], device=device)
    att = torch.as_tensor(query_att[start:end],device=device)
    embs = my_encoder.get_emb(inp, att, use_amp=True, use_no_grad=True)
    
    del inp, att, embs

In [ ]:
train_queries[14848]

query_inputs[14848]

In [ ]:
(tokens_size, max_length ) = tokens_paths.queries_shape
N = tokens_size

query_inputs = dataset.queries_input_ids
query_att = dataset.queries_attention_mask

start = 14819
end = 14821
print(f"start:end: {start}-{end}")
inp  = torch.as_tensor(query_inputs[start:end], device=device)
att = torch.as_tensor(query_att[start:end],device=device)
embs = my_encoder.get_emb(inp, att, use_amp=True, use_no_grad=True)


In [ ]:
14820
tokens = tokenizer.convert_ids_to_tokens(query_inputs[14820])

tokens


In [ ]:
cui = 'C1257890'
queries = []
for q in train_queries:
    if q[1] == cui and q[2].find("[ME]") == -1:
        enc = tokenizer(q[2], padding="max_length", truncation=True, max_length=75)
        mention_end_positions = (query_inputs[i] == mention_end_token_id).nonzero()[0]
        queries.append(q)


In [ ]:
for idx, q in enumerate(queries):
    enc = tokenizer(q[2], padding="max_length", truncation=True, max_length=75)
    print(f"idx in queries: {idx}")
    (torch.as_tensor(enc["input_ids"]) ==  mention_end_token_id).nonzero()[0]



In [ ]:
 'iovascular com orbid', '##ities', 'in', 'patients', 'with', 'ps', '##oria', '##sis', ':', 'a', 'k', '##ore', '##an', 'nationwide', 'population', '-', 'based', 'co', '##hor', '##t', 'study', 'there', 'is', 'a', 'lack', 'of', 'nationwide', 'studies', 'examining', 'the', 'e', '##pid', '##em', '##iology', 'and', 'com', '##or', '##bid', '##ities', 'of', 'ps', '##oria', '##sis', 'v', '##ul', '##gar', '##is', '(', 'ps', '##v', ')', 'and', 'ps', '##oria', '##tic', 'art', '##hr', '##itis', '(', 'ps', '##a', ')', 'in', 'as', '##ian', '[MS]', 'populations', '[SEP]']